## Setting  Up Dictionary

Functions to set up a vocab of words and perform simple preprocessing on the given dataset

In [74]:
import torch
import gensim 
import numpy as np

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)
    
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()
        # words_found += 1
    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
        with open(path, 'r') as f:
            lines = f.readlines()
        tokens = 0
        # process sentences to tokens
        processedLines = [gensim.utils.simple_preprocess(sentence,min_len=1) for sentence in lines]
        #create word list from token using utf8 encoding
        for words in processedLines : 
            tokens += len(words) 
            for word in words : 
                self.dictionary.add_word(word)  
        
        # Tokenize the file content
        ids = np.zeros(tokens)
        token = 0
        with open(path, 'r') as f:
            lines = f.readlines()
        processedLines = [gensim.utils.simple_preprocess(sentence,min_len=1) for sentence in lines]
        #create word list from token using utf8 encoding
        for words in processedLines :
            for word in words:
                ids[token] = int(self.dictionary.word2idx[word])
                token += 1
        
        return ids
        

In [75]:
UNK_TOKEN = "<UNK>"

## Glove Embeddings
using pretrained glove embeddings to map words to vectors 

In [76]:
#setup Glove word embeddings
with open('glove.6B/glove.6B.50d.txt', 'r') as f:
    lines = f.readlines()

glove = dict()

for line in lines:
    items = line.split()
    word = items[0]
    vector = torch.FloatTensor(list(map(float, items[1:])))
    glove[word] = vector

## Setting Up Embeddings In Pytorch
  

In [77]:
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# make dictionary
corpus = Corpus()
ids = corpus.get_data('./brown.txt', 20)
corpus.dictionary.add_word(UNK_TOKEN)
vocab_size = len(corpus.dictionary)

print(vocab_size)
#weights for embedding layer

weights_matrix = torch.zeros(vocab_size, 50)
words_found = 0

for i in range(vocab_size-1):
    # print(corpus.dictionary.idx2word[word])
    try:
        weights_matrix[i] = glove[corpus.dictionary.idx2word[i]]
        words_found += 1
    except KeyError:
        weights_matrix[i] = torch.from_numpy(np.random.normal(scale=0.6, size=(50, )))
weights_matrix[vocab_size-1] =  torch.mean(weights_matrix, 0)

cpu
41455


In [91]:
ids[0:10]



10.0

## Q1 Neural Network Model

In [5]:
# Embedding layer
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim


# Neural Network Model

class language_model(nn.Module):
    def __init__(self, weights_matrix, hidden_size, vocab_size):
        super(language_model, self).__init__()
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
        self.hidden_size = hidden_size
        self.linear = nn.Linear( 200,hidden_size)
        self.linear2 = nn.Linear( hidden_size, vocab_size)
        self.sf = nn.Softmax(dim=0)
       
    def forward(self, inp):
        out = self.embedding(inp)
        out1 = torch.concat([out[i] for i in range(out.size(0))])
        out2 = self.linear(out1)
        out3 = self.linear2(out2)
        out4 = self.sf(out3)
        return out4
        # return self.gru(self.embedding(inp), hidden)
    
    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))

In [64]:
model = language_model(weights_matrix, hidden_size=300, vocab_size=vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 5
seq_length = 4

In [96]:
for epoch in range(epochs):
    
    for i in range(0, len(ids) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[i:i+seq_length]
        inputs = torch.tensor(inputs, dtype=torch.long).to(device)
        output = model(inputs)
        real = np.zeros(output.shape)
        real[int(ids[i+seq_length])] = 1
        real = torch.tensor(real, dtype=torch.float).to(device)
        loss = criterion(output,real )
        
        
        # Backward and optimize
        model.zero_grad()
        loss.backward()
        optimizer.step()
        step = (i+1) // seq_length
        if step % 500 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, 5, step, (10000), loss.item(), np.exp(loss.item())))

        # targets = ids[:, (i+1):(i+1)+seq_length].to(device)
    
        # x = []
        # for word in inputs:
        #     try:
        #         x.append(corpus.dictionary.word2idx[word.lower()])
        #     except:
        #         x.append(corpus.dictionary.word2idx[UNK_TOKEN])

        # x = torch.LongTensor(x)
        
        # output


# print(output.size())
# corpus.dictionary.idx2word[int(torch.argmax(output).data)]

Epoch [1/5], Step[0/10000], Loss: 10.6324, Perplexity: 41455.89
Epoch [1/5], Step[500/10000], Loss: 10.6324, Perplexity: 41458.26
Epoch [1/5], Step[1000/10000], Loss: 10.6324, Perplexity: 41458.06


KeyboardInterrupt: 

In [80]:
ids[0:4]

array([0., 1., 2., 3.])